In [ ]:
from pdbfixer import PDBFixer
from openmm.app import PDBFile

In [ ]:
fixer = PDBFixer("./4xkl.pdb")

In [ ]:
for chain in list(fixer.topology.chains()):
    print(chain.id)

In [ ]:
fixer.removeChains([2, 3, 4, 6, 7, 8, 9, 10, 11])
fixer.findMissingResidues()


# only add missing residues in the middle of the chain, do not add terminal ones
chains = list(fixer.topology.chains())
keys = fixer.missingResidues.keys()
missingResidues = dict()
for key in keys:
    chain = chains[key[0]]
    if not (key[1] == 0 or key[1] == len(list(chain.residues()))):
        missingResidues[key] = fixer.missingResidues[key]
fixer.missingResidues = missingResidues


fixer.findMissingAtoms()
fixer.addMissingAtoms()


PDBFile.writeFile(fixer.topology, fixer.positions, open("4xkl_fixed.pdb", 'w'))

In [ ]:
from openmm import app, unit

pdb = app.PDBFile("4xkl_fixed.pdb")
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

In [ ]:
# use app.Modeller to add hydrogens and solvent
modeller = app.Modeller(pdb.topology, pdb.positions)

In [ ]:
# delete acetate ions
toDelete = [r for r in pdb.topology.residues() if r.name == "ACT"]
modeller.delete(toDelete)

In [ ]:
# use app.Modeller to add hydrogens and solvent
modeller.addHydrogens(forcefield)
modeller.addSolvent(forcefield, model='tip3p', padding=1.0*unit.nanometers)
app.PDBFile.writeFile(modeller.topology, modeller.positions, open("4xkl_modeller_tip3p.pdb", 'w'))